In [1]:
from module.fetching_credentials import fetching_credentials
from module.greathr_logger import greathr_logger
from module.message_sender import message_sender
from dotenv import dotenv_values
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pytz

def main():
    credentials = fetching_credentials('.env.secret')
    users = list(credentials.keys())
    for user in users:
        status = 0
        for _ in range(3):
            if status:
                continue
            options = Options()
            options.add_argument("--headless")           # Run in headless mode
            options.add_argument("--disable-dev-shm-usage")
            driver = webdriver.Chrome(options=options)
            try:
                greathr_logger(driver,credentials[user]['username'],credentials[user]['password'])
                time.sleep(5)
                driver.quit()
                print('Successfully logged')
                status+=1
            except Exception as e:
                error_message = e
                time.sleep(5)
                driver.quit()
                time.sleep(5)

            if status:
                webhook_url = dotenv_values('.env.shared')['WEBHOOK']
                ist = pytz.timezone('Asia/Kolkata')
                current_ist = datetime.now(ist).strftime("%Y-%m-%d %H:%M:%S")
                message = f"{current_ist}:Logger action completed for {user}"
                message_sender(message,webhook_url)
        if not status:
            webhook_url = dotenv_values('.env.shared')['WEBHOOK']
            ist = pytz.timezone('Asia/Kolkata')
            current_ist = datetime.now(ist).strftime("%Y-%m-%d %H:%M:%S")
            message = f"{current_ist}:Error while logging for {user} \n Error:{error_message}"
            message_sender(message,webhook_url)

